# Creating a feature matrix from a DB Query

In [2]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [3]:
#load in data base
db = pd.read_csv('/Users/shayneufeld/GitHub/mouse_bandit/session_record.csv',index_col=0)

### Query all 80-20 sessions where performance > 0.7 and block structure was 50

In [24]:
r = db[((db['Left Reward Prob'] == 0.7) |  (db['Right Reward Prob'] == 0.7))].copy()
r = r[r['p(high Port)'] > 0.6].copy()
r = r[r['Block Range Min'] == 50].copy()
session_names = r['Session ID'].values

In [25]:
r

,Session ID,Mouse ID,Date,Phase,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
73,11102016_q43,q43,11102016,2.0,0.7,0.3,50.0,50.0,506.0,6.0,306.0,0.72,2.0,1.0,35.0,35.0
110,11102016_q45,q45,11102016,2.0,0.7,0.3,50.0,50.0,314.0,3.0,184.0,0.75,2.0,1.0,35.0,35.0
168,07252016_quirrel,quirrel,7252016,2.0,0.7,0.3,50.0,50.0,632.0,7.0,375.0,0.82,2.0,1.0,45.0,40.0
169,07252016_sprout,sprout,7252016,2.0,0.3,0.7,50.0,50.0,710.0,8.0,429.0,0.71,2.0,1.0,45.0,40.0
175,07262016_sprout,sprout,7262016,2.0,0.7,0.3,50.0,50.0,597.0,7.0,361.0,0.75,2.0,1.0,45.0,40.0
176,07262016_tom,tom,7262016,2.0,0.3,0.7,50.0,50.0,598.0,6.0,347.0,0.75,2.0,1.0,45.0,40.0
177,07262016_tonks,tonks,7262016,2.0,0.7,0.3,50.0,50.0,612.0,7.0,353.0,0.77,2.0,1.0,45.0,40.0
178,07262016_volde,volde,7262016,2.0,0.7,0.3,50.0,50.0,424.0,4.0,250.0,0.74,2.0,1.0,45.0,40.0
179,07272016_dumble,dumble,7272016,2.0,0.3,0.7,50.0,50.0,343.0,4.0,207.0,0.80,2.0,1.0,40.0,40.0
180,07272016_harry,harry,7272016,2.0,0.3,0.7,50.0,50.0,537.0,6.0,329.0,0.75,2.0,1.0,45.0,40.0


In [26]:
r.shape

(34, 16)

## load in csv files (from running exportTrials.m)

In [27]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

root_dir = '/Users/shayneufeld/GitHub/mouse_bandit/data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [28]:
len(trial_df)

34

## convert into 1 feature matrix

In [29]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [30]:
master_matrix.shape

(17856, 50)

In [31]:
master_matrix['Higher p port'].mean()

0.75128808243727596

## Save combined feature matrix  

In [32]:
master_matrix.to_csv(os.path.join(root_dir,'full_7030_02192017.csv'))